In [20]:
import turicreate as tc
import pandas as pd

In [22]:
data = pd.read_csv("merged_final.csv")

In [23]:
#Comments for this cell are in Book_Rec_Project_notebook1
data2=data[['title','rating','user_id']]
data3=data2.groupby('title')[['rating','user_id']].agg('count')
mask=data3['rating']>=10
data4=data3[mask]
merged_inner=pd.merge(left=data4,right=data2,left_on='title',right_on='title',how='inner')
Final=merged_inner[['title','rating_y','user_id_y']]
Final.to_csv('Marketing_Final_Proj.csv',index=False)

In [24]:
#Using Turicreate as a recommendation system
books = tc.SFrame.read_csv("Marketing_Final_Proj.csv", header=True,delimiter=',')
books

Finished parsing file /Users/Thiru/Desktop/Marketing Analytics/Marketing_Final_Proj.csv

Parsing completed. Parsed 100 lines in 1.2581 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/Thiru/Desktop/Marketing Analytics/Marketing_Final_Proj.csv

Parsing completed. Parsed 637070 lines in 0.705206 secs.

Columns:
	title	str
	rating_y	int
	user_id_y	str

Rows: 637070

Data:
+-------------------------------+----------+-------------------------------+
|             title             | rating_y |           user_id_y           |
+-------------------------------+----------+-------------------------------+
| "A" is for Abigail: An Alm... |    0     | bb88527d0cf05352400e90f7da... |
| "A" is for Abigail: An Alm... |    0     | 4a9cc023eed8850449791aadfd... |
| "A" is for Abigail: An Alm... |    4     | bcd69f2759636f3786639d846c... |
| "A" is for Abigail: An Alm... |    3     | 51f2623b0a4a7ef500aa08d2a2... |
| "A" is for Abigail: An Alm... |    1     | 8e7e5b546a63cb9add8431ee69... |
| "A" is for Abigail: An Alm... |    0     | 891c1011cef0d56dd5b448dd8c... |
| "A" is for Abigail: An Alm... |    0     | bb88527d0cf05352400e90f7da... |
| "A" is for Abigail: An Alm... |    0     | 4a9cc023eed8850449791aadfd... |
| "A" is for Abigail: An Alm... |    4     | bcd69f2759636f3786639d846c... |
| "A" is for Abigail: An Alm... |    3     | 51f2623b0a4a7ef500aa08d2a2... |
+-------------------------------+----------+-------------------------------+
[637070 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [25]:
books.show()

Materializing SFrame

In [26]:
training_data, validation_data = tc.recommender.util.random_split_by_user(books, 'user_id_y', 'title',item_test_proportion=0.3)


In [28]:
model = tc.recommender.item_similarity_recommender.create(training_data,
user_id='user_id_y', item_id='title',target='rating_y') 
items_similarity = model.get_similar_items()

Preparing data set.

Data has 632237 observations with 37574 users and 13957 items.

Data prepared in: 1.05448s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 1.75s                               | 100              | 13957           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.81257s

In [29]:
model.evaluate(validation_data)
#Item based Collaborative filtering Overall RMSE: 2.981362699909857

/opt/anaconda3/lib/python3.7/site-packages/turicreate/aggregate.py:91: UserWarning: Passing parameter(s) to COUNT is deprecated. This functionality will be removed in the next major release. Any passed parameters are ignored.
  warnings.warn('Passing parameter(s) to COUNT is deprecated. This functionality will be removed in '



Precision and recall summary statistics by cutoff
+--------+---------------------+---------------------+
| cutoff |    mean_precision   |     mean_recall     |
+--------+---------------------+---------------------+
|   1    | 0.19306184012066357 | 0.04031073460019136 |
|   2    | 0.15761689291101053 | 0.05879965515162694 |
|   3    | 0.13574660633484165 |  0.0785393171795611 |
|   4    | 0.12405731523378583 |  0.1009382350147086 |
|   5    | 0.11402714932126698 | 0.11317187270973593 |
|   6    | 0.10859728506787335 | 0.12417058081679504 |
|   7    |  0.1029950441715148 |  0.1333218952555637 |
|   8    | 0.09803921568627451 |  0.1443942893600805 |
|   9    | 0.09250879839115134 | 0.15235775468913895 |
|   10   |  0.0859728506787331 |  0.1555865510171409 |
+--------+---------------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 2.981362699909857

Per User RMSE (best)
+-------------------------------+------+-------+
|           user_id_y           | rmse | count |
+--

{'precision_recall_by_user': Columns:
 	user_id_y	str
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 11934
 
 Data:
 +-------------------------------+--------+--------------------+--------+-------+
 |           user_id_y           | cutoff |     precision      | recall | count |
 +-------------------------------+--------+--------------------+--------+-------+
 | 02538800d798d36723b3cef788... |   1    |        1.0         | 0.125  |   8   |
 | 02538800d798d36723b3cef788... |   2    |        0.5         | 0.125  |   8   |
 | 02538800d798d36723b3cef788... |   3    | 0.3333333333333333 | 0.125  |   8   |
 | 02538800d798d36723b3cef788... |   4    |        0.5         |  0.25  |   8   |
 | 02538800d798d36723b3cef788... |   5    |        0.4         |  0.25  |   8   |
 | 02538800d798d36723b3cef788... |   6    | 0.3333333333333333 |  0.25  |   8   |
 | 02538800d798d36723b3cef788... |   7    | 0.2857142857142857 |  0.25  |   8   |
 | 02538800d798d36723b3cef788... |   8    | 

In [31]:
model = tc.recommender.ranking_factorization_recommender.create(training_data,
user_id='user_id_y', item_id='title',target='rating_y')
results = model.recommend(k=3)

Preparing data set.

Data has 632237 observations with 37574 users and 13957 items.

Data prepared in: 0.846274s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

recommendations finished on 33000/37574 queries. users per second: 9886.37

recommendations finished on 34000/37574 queries. users per second: 9916.06

recommendations finished on 35000/37574 queries. users per second: 9992.03

recommendations finished on 36000/37574 queries. users per second: 10079

recommendations finished on 37000/37574 queries. users per second: 10137.9

In [32]:
model.evaluate(validation_data)
#Matrix factorization Overall RMSE: 2.438094020279795


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.03469079939668176  | 0.010377219659053099 |
|   2    | 0.03996983408748114  | 0.02867989231190058  |
|   3    | 0.03871292106586226  | 0.03504681070376372  |
|   4    | 0.03582202111613873  | 0.04245109935519196  |
|   5    | 0.034389140271493174 | 0.04957217254125299  |
|   6    | 0.03418803418803418  | 0.05665484916790731  |
|   7    | 0.03275156216332686  | 0.06076751162898606  |
|   8    | 0.032051282051282055 | 0.06684852435404104  |
|   9    | 0.03150661974191382  | 0.07230000728914078  |
|   10   | 0.031221719457013585 | 0.08061121979371375  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 2.438094020279795

Per User RMSE (best)
+-------------------------------+----------------------+-------+
|       

{'precision_recall_by_user': Columns:
 	user_id_y	str
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 11934
 
 Data:
 +-------------------------------+--------+---------------------+--------+-------+
 |           user_id_y           | cutoff |      precision      | recall | count |
 +-------------------------------+--------+---------------------+--------+-------+
 | 02538800d798d36723b3cef788... |   1    |         0.0         |  0.0   |   8   |
 | 02538800d798d36723b3cef788... |   2    |         0.5         | 0.125  |   8   |
 | 02538800d798d36723b3cef788... |   3    |  0.3333333333333333 | 0.125  |   8   |
 | 02538800d798d36723b3cef788... |   4    |         0.25        | 0.125  |   8   |
 | 02538800d798d36723b3cef788... |   5    |         0.2         | 0.125  |   8   |
 | 02538800d798d36723b3cef788... |   6    | 0.16666666666666666 | 0.125  |   8   |
 | 02538800d798d36723b3cef788... |   7    |  0.2857142857142857 |  0.25  |   8   |
 | 02538800d798d36723b3cef788... |